# Project forecasting

## Imports

In [1]:
from IPython.core.display_functions import display
from scipy.stats import chi2

# Database
from models.logit_lmpc12_model0 import database

# Model pref
from models.logit_lmpc12_model4 import nests 
from models.logit_lmpc12_model3 import V_boxcox as V_pref 
from models.logit_lmpc12_model3 import chosen_alternative as choice_pref

# biogeme
import biogeme.biogeme as bio
from biogeme.models import lognested, logcnl

# market share
from biogeme.expressions import Expression
from typing import NamedTuple


Informations about the data [here](https://transp-or.epfl.ch/documents/technicalReports/CS_LPMC.pdf).

## Functions

In [ ]:
class IndicatorTuple(NamedTuple):
    """Tuple storing the value of an indicator, and the bounds on its confidence interval."""

    value: float
    lower: float
    upper: float



In [ ]:
def market_share(utilities: dict[int, Expression]) -> dict[str, IndicatorTuple]:
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utilities: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.

    :return: a dictionary where each entry corresponds to an
        alternative, and associates its name with the IndicatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.


    """
    prob_pt = logit(utilities, None, 0)
    prob_car = logit(utilities, None, 1)
    prob_sm = logit(utilities, None, 2)

    simulate = {
        'weight': normalizedWeight,
        'Prob. PT': prob_pt,
        'Prob. car': prob_car,
        'Prob. SM': prob_sm,
    }
    biosim = BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results.get_beta_values())

    # We also calculate confidence intervals for the calculated quantities
    betas = biogeme.free_beta_names
    b = results.get_betas_for_sensitivity_analysis(betas)
    left, right = biosim.confidence_intervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities

    # Alternative car
    simulated_values['Weighted choice_prob. car'] = (
        simulated_values['weight'] * simulated_values['Prob. car']
    )
    left['Weighted choice_prob. car'] = left['weight'] * left['Prob. car']
    right['Weighted choice_prob. car'] = right['weight'] * right['Prob. car']

    market_share_car = simulated_values['Weighted choice_prob. car'].mean()
    market_share_car_left = left['Weighted choice_prob. car'].mean()
    market_share_car_right = right['Weighted choice_prob. car'].mean()

    # Alternative public transportation
    simulated_values['Weighted choice_prob. PT'] = (
        simulated_values['weight'] * simulated_values['Prob. PT']
    )
    left['Weighted choice_prob. PT'] = left['weight'] * left['Prob. PT']
    right['Weighted choice_prob. PT'] = right['weight'] * right['Prob. PT']

    market_share_pt = simulated_values['Weighted choice_prob. PT'].mean()
    market_share_pt_left = left['Weighted choice_prob. PT'].mean()
    market_share_pt_right = right['Weighted choice_prob. PT'].mean()

    # Alternative slow modes
    simulated_values['Weighted choice_prob. SM'] = (
        simulated_values['weight'] * simulated_values['Prob. SM']
    )
    left['Weighted choice_prob. SM'] = left['weight'] * left['Prob. SM']
    right['Weighted choice_prob. SM'] = right['weight'] * right['Prob. SM']

    market_share_sm = simulated_values['Weighted choice_prob. SM'].mean()
    market_share_sm_left = left['Weighted choice_prob. SM'].mean()
    market_share_sm_right = right['Weighted choice_prob. SM'].mean()

    return {
        'Car': IndicatorTuple(
            value=market_share_car,
            lower=market_share_car_left,
            upper=market_share_car_right,
        ),
        'Public transportation': IndicatorTuple(
            value=market_share_pt,
            lower=market_share_pt_left,
            upper=market_share_pt_right,
        ),
        'Slow modes': IndicatorTuple(
            value=market_share_sm,
            lower=market_share_sm_left,
            upper=market_share_sm_right,
        ),
    }



## Goal

1. Report the market shares predicted by Modelpref for each scenario.
Do they match your expectations? Compare those with the original market shares. [1 point]

## Scenario 1

In [2]:
from models.logit_lmpc12_model5 import V_boxcox as V_1
from models.logit_lmpc12_model5 import chosen_alternative as choice_1

An additional charge of £1.5 to the car users

In [5]:
# model pref = nested model
logprob_nested = lognested(V_1, None,nests, choice_1)
biogeme_nested = bio.BIOGEME(database, logprob_nested)
biogeme_nested.modelName = 'model_scenario1'
results_scenario1 = biogeme_nested.estimate(recycle=False)

In [7]:
display(results_scenario1.get_estimated_parameters())

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_fare,-0.075303,0.034547,-2.179720,0.029278
beta_travel_time,-8.747491,0.990511,-8.831292,0.000000
beta_travel_time_2,-2.314073,0.606624,-3.814677,0.000136
beta_travel_time_2_non-work-education-related,0.091320,0.171649,0.532018,0.594713
beta_travel_time_3,-0.900950,0.427374,-2.108108,0.035022
beta_travel_time_3_non-work-education-related,-0.244121,0.110973,-2.199824,0.027819
beta_travel_time_4,-1.230525,0.603993,-2.037317,0.041618
beta_travel_time_4_non-work-education-related,-0.330736,0.131628,-2.512660,0.011982
beta_travel_time_non-work-education-related,0.117226,0.552344,0.212233,0.831925
constant_2,-8.612661,0.967823,-8.899003,0.000000


## Scenario 2

In [4]:
from models.logit_lmpc12_model6 import V_boxcox as V_2
from models.logit_lmpc12_model6 import chosen_alternative as choice_2

A decrease of the public transport cost by 20%.

In [6]:
# model pref = nested model
logprob_nested = lognested(V_2, None,nests, choice_2)
biogeme_nested = bio.BIOGEME(database, logprob_nested)
biogeme_nested.modelName = 'model_scenario2'
results_scenario2 = biogeme_nested.estimate(recycle=False)